In [ ]:
import sys
import json
import random
from PIL import Image, ImageDraw

# le user choisit le fichier qu'il veut (json) et map quelle est la colonne année et puissance (valeur) à étudier et aussi le nom du fichier de sortie
nomFichier = input("Nom du fichier texte à lire (ex: fichier.json) : ").strip()
colAnnée = input("Nom de la colonne des années dans ton dataset :").strip()
colPuissance = input("Nom de la colonne autre dans ton dataset (ex : mm de précipitations, température moyenne):").strip()
nomDesortie = input("Nom du fichier de sortie (ex : mavisualisation): ").strip()

# gestion des erreurs relatives au json, potentiellement mal encodé ou mal renseigné par le user
try:
    with open(nomFichier, "r", encoding="utf-8") as f:
        data = json.load(f)
except FileNotFoundError:
    print(f"Le fichier '{nomFichier}' n'existe pas.")
    sys.exit(1)
except json.JSONDecodeError:
    print("Erreur : le fichier n'est pas un JSON valide.")
    sys.exit(1)

# mapping des données avec les variables définies en input par le user
annees = [d[colAnnée] for d in data]
valeurs = [d[colPuissance] for d in data]

# normalisation pour avoir une meilleure lisibilité des données (cela évite que si on a une valeur à 100 et une a 5000000 on en voit pas une des deux)
val_min = min(valeurs)
val_max = max(valeurs)

def normaliser(val):
    if val_max == val_min:
        return 0.5
    return (val - val_min) / (val_max - val_min)

scales = [normaliser(v) for v in valeurs]

# définition du canevas sur lequel on va insérer les pixels
hauteur = 1024
largeur = len(annees) * 25  # espacement horizontal
img = Image.new("RGB", (largeur, hauteur), color=(0, 0, 0))  # fond noir
draw = ImageDraw.Draw(img)

#paramètre des cercles qui seront nos étoiles (ici il y a jaune mix et max pour créer un dégradé selon l'intensité, sa valeur dans la normalisaiton)
rayon_min = 5
rayon_max = 25
jaune_max = (255, 255, 0)
jaune_min = (128, 128, 0)
dispersion = hauteur // 2  # permet aux cercles d'être répartis sur toute la hauteur

# Dessiner les cercles (étoiles)
for i, s in enumerate(scales):
    # Position horizontale
    x = i * 25 + 12  # centré sur la case
    # Position verticale aléatoire autour du centre
    y_central = hauteur // 2
    y = int(y_central + random.uniform(-dispersion, dispersion))
    
    # Taille du cercle (ici s c'est la valeur noramlisée entre 0 et 1) en gros on utilise S pour faire un coef sur le rayon mix et max définis en dur avant
    rayon = int(rayon_min + s * (rayon_max - rayon_min))
    
    # couleur du cercle : on utilise jaune min et jaune max comme pour les tailles du rayon et on utilise S pour définir l'intensité, proche de 0 sera terne et proche de 1 sera flash
    r = int(jaune_min[0] * (1 - s) + jaune_max[0] * s)
    g = int(jaune_min[1] * (1 - s) + jaune_max[1] * s)
    b = int(jaune_min[2] * (1 - s) + jaune_max[2] * s)
    couleur = (r, g, b)
    
    # dessiner le cercle
    draw.ellipse(
        [x - rayon, y - rayon, x + rayon, y + rayon],
        fill=couleur
    )

#sauvegarde du fichier dans le répertoire donné 
nom_image =  f"{nomDesortie}.png"
img.save(nom_image)
print(f"Visualisation sauvegardée -> '{nom_image}'")


Visualisation sauvegardée -> 'monvisualisation.png'
